In [1]:
from datasaurus import *
from fengineer import *

In [2]:
#Data Formatting Params
stem = 'stem'
english = True

#Feature Engineering Params
tf_idf_rate = 0.2
pca_num = 50

data_params={'stem': stem, 'english': english, 'tf_idf_rate': tf_idf_rate, 'pca_num': pca_num}

In [3]:
data = transform_data(sample_data(), stem=stem, english=english)

/Users/flatironschool/Documents/module3_project/jokes/jokes/datasaurus.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  funnies['label'] = 1
/Users/flatironschool/Documents/module3_project/jokes/jokes/datasaurus.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  duds['label'] = 0


In [4]:
fengineer = Fengineer(tf_idf_rate=tf_idf_rate, pca_num=pca_num)

In [5]:
data_feng = fengineer.engineer_features(data)

In [6]:
target = data_feng['_label']
features = data_feng.drop('_label', axis=1)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [8]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))

In [9]:
from sklearn.model_selection import GridSearchCV
import os

#return the fitted grid search...
#model is non-parameterized model to grid search, params are the params of interest
#filename is a file to write results to in csv format

#TODO: edit so it writes more metrics? idk. 
def write_results(results, directory, data_params, filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    df = pd.DataFrame.from_dict(dict(results), orient="index")
    data_param_string = '_'.join([str(x) + '-' + str(data_params[x]) for x in data_params.keys()])
    df.to_csv(directory + filename + '_' + data_param_string + '.csv')

In [10]:
#Use this function to perform grid search! It will automatically write to 'models/' directory.
#NOTE: data_params is just the values we used to build the data and do feature engineering... 
#it gets passed in because we need to make a note of it in the file where the outputs for the model go.

def grid_search(model, x_train, y_train, param_grid, data_params, filepath='models/'):
    clf = model
    gs_tree = GridSearchCV(clf, param_grid, cv=5)
    gs_tree.fit(x_train, y_train)
    results = gs_tree.cv_results_
    model_str = str(model)
    model_str = model_str[0:model_str.find('(')]
    write_results(results, filepath + model_str + '/', data_params, str(param_grid))
    return results

In [20]:
data_feng.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,_label
5tyqag,2.922102,1.546675,-4.565186,-2.415032,4.903563,2.117861,-0.113742,1.495028,-0.297710,-0.549989,...,-0.277125,0.337885,-0.101205,0.417163,0.103463,-0.002004,-0.132312,0.017878,-0.008045,1
5txxsu,-0.888196,0.035620,0.250953,-0.098151,0.048721,0.014222,0.049569,0.000237,-0.011350,0.038969,...,-0.020590,0.099165,0.060755,-0.047182,0.010591,0.017745,-0.051926,-0.016032,0.045021,1
5tx5ur,-0.333514,0.183209,-0.155094,-0.174504,0.023125,-0.135158,-0.368292,-0.204076,0.008149,-0.167633,...,0.207539,0.005172,0.002223,-0.105697,0.060912,0.027206,-0.124054,-0.031305,-0.055178,1
5twufa,1.338863,-2.121422,0.841189,-0.192709,-0.035449,-0.179740,-0.377715,0.115555,0.207197,-0.925741,...,0.118350,-0.197184,-0.035026,0.103808,-0.067812,0.005638,0.151855,0.111031,0.160864,1
5twqzh,-0.886488,0.034237,0.261263,-0.090178,0.038020,-0.000824,0.052063,-0.011013,-0.011288,0.037753,...,-0.016688,0.011615,-0.076655,0.098216,0.033823,0.034235,0.040367,0.018773,0.057034,1


In [36]:
print_metrics(nb_classifier.predict(x_test), y_test)

Precision Score: 0.30152604806174355
Recall Score: 0.7118012422360248
Accuracy Score: 0.5865665046398586
F1 Score: 0.4236076885165106


In [42]:
# Multinomial Naive Bayes GridSearch - SCOTT: DO THIS ONE, choose some params, follow KNN Grid Search cell 
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(x_train, y_train)
# print_metrics(nb_classifier.predict(x_test), y_test)
param_grid = {
    'priors': []
    'var_smoothing':[1e-09]
}
results = grid_search(nb_classifier, x_train, y_train, param_grid, data_params)

In [24]:
# KNN Grid Search
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [1], 
    'p': [2], 
    'n_jobs': [5]
}
results = grid_search(knn_clf, x_train, y_train, param_grid, data_params)

In [ ]:
# Random Forest Grid Search - SCOTT: DO THIS ONE, choose some params, follow KNN Grid Search cell 
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
param_grid = {
    'n_estimators': [100]
}
results = grid_search(rf_classifier, x_train, y_train, param_grid, data_params)

In [ ]:
# SVM Grid Search
from sklearn import svm

svm_clf = svm.SVC(kernel='linear')
param_grid = {
    'kernel': ['linear'],
    'degree': [1]   
}
results = grid_search(svm_clf, x_train, y_train, param_grid, data_params)

In [ ]:
#XGBooster
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()
param_grid = {
    'max_depth': [5],
    'n_estimators': [100],
    'n_jobs': [2]    
}
results = grid_search(xgb_clf, x_train, y_train, param_grid, data_params)